In [52]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_sampled_debug'
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'input_results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'input_results', 'run', run_name))
        run = (json.load(json_file))
        if 'index_stats' not in run['result']:
            continue
        for run_index_stats in run['result']['index_stats']:
            run_index_stats['dataset'] = get_dataset_from_testcase(test_case)
            index_stats.append(run_index_stats)
idf = pd.json_normalize(index_stats)
print(idf.columns)

Index(['index_build_duration', 'index_load_duration', 'index_name',
       'index_size', 'dataset'],
      dtype='object')


In [53]:
epsilon = {
    "sampledPgm": 16 * 128 * 2,
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree256": 256,
    "btree1024": 1024,
    "btree2048": 2048,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "sampledPgm" in x:
        return "SAMPLED_PGM"
    elif "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"
idf['epsilon'] = idf['index_name'].map(lambda x: epsilon[x])
idf['index_type'] = idf['index_name'].map(lambda x: indexType(x))
# Per dataset
rows = duckdb.sql('SELECT * FROM idf')
print(rows)


┌──────────────────────┬─────────────────────┬─────────────┬────────────┬─────────┬─────────┬─────────────┐
│ index_build_duration │ index_load_duration │ index_name  │ index_size │ dataset │ epsilon │ index_type  │
│        int64         │        int64        │   varchar   │   int64    │ varchar │  int64  │   varchar   │
├──────────────────────┼─────────────────────┼─────────────┼────────────┼─────────┼─────────┼─────────────┤
│              3940440 │           183833035 │ sampledPgm  │       1112 │ fb      │    6400 │ SAMPLED_PGM │
│            585667926 │           235303338 │ pgm256      │     194376 │ fb      │     256 │ PGM         │
│            475671605 │           232060569 │ pgm1024     │      16768 │ fb      │    1024 │ PGM         │
│            438000421 │           282731222 │ pgm2048     │       4576 │ fb      │    2048 │ PGM         │
│            590052782 │           293945725 │ flatpgm256  │     193680 │ fb      │     256 │ FLAT_PGM    │
│            474082307 │    

In [54]:
duckdb.sql("SELECT index_name, index_build_duration, index_size FROM idf " 
"WHERE index_type=='PGM' OR index_type=='SAMPLED_PGM' OR index_type=='BTREE'")

┌────────────┬──────────────────────┬────────────┐
│ index_name │ index_build_duration │ index_size │
│  varchar   │        int64         │   int64    │
├────────────┼──────────────────────┼────────────┤
│ pgm256     │            585667926 │     194376 │
│ pgm1024    │            475671605 │      16768 │
│ pgm2048    │            438000421 │       4576 │
│ sampledPgm │              3940440 │       1112 │
│ btree256   │               184388 │    1211256 │
│ btree1024  │                48635 │     304872 │
│ btree2048  │                22728 │     156552 │
└────────────┴──────────────────────┴────────────┘